# Baseline delays and rates

[L. Blackburn, Sep 2018; rewritten for Python 3.9, Dec 2022]

The purpose of this test is to check continuity of delay and rate solutions. Fringe delay and rate should be relatively smooth from scan to scan, and outliers indicate some issue with the fringe solution.

In [ ]:
alistf = 'alist.v6'

In [ ]:
# basic import and helper functions
import pandas as pd
from eat.io import hops, util
from eat.hops import util as hu
from eat.plots import util as pu
from collections import OrderedDict
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.ticker import MultipleLocator
import seaborn as sns
sns.reset_orig()
# sns.set_palette(sns.color_palette(sns.hls_palette(16, l=.6, s=.6)))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_formats=['svg']
import sys
nb_stdout = sys.stdout # grab for later
def wide(w=8, h=3): plt.setp(plt.gcf(), figwidth=w, figheight=h); \
    plt.tight_layout()
def tightx(): plt.autoscale(enable=True, axis='x', tight=True)
def multline(xs, fun=plt.axvline):
    for x in xs: fun(x, alpha=0.25, ls='--', color='k')
def toiter(x):
    return(x if hasattr(x, '__iter__') else [x,])
# pd.options.display.float_format = '{:,.6f}'.format
from IPython.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))
import string

In [ ]:
# define and load data
#stage = '3.+adhoc'
#datadir = f'/home/iniyan/github/2021-april/dev-template/hops-b1/{stage}/data/'
datadir = os.environ['DATADIR']
a = util.noauto(hops.read_alist(os.path.join(datadir, alistf)))
util.fix(a)
util.unwrap_mbd(a)
util.add_days(a)
util.add_delayerr(a)
util.add_path(a)
util.add_scanno(a)

In [ ]:
# data filters
a = a[(a.snr > 7.) & ~a.baseline.isin({'RS', 'SR'})].copy()

In [ ]:
# expt_no boundary ticker marks vs scan
elines = a.scan_no.sort_values().values[np.nonzero(np.diff(a.expt_no.sort_values()) > 0)[0]] + 0.5

In [ ]:
sites = set(''.join(a.baseline))
for (i, site) in enumerate(sites):
    hu.trendplot(a, site, vlines=elines, col='mbd_unwrap')
    plt.title('measured delays [ns @ %.0f MHz]' % (a.iloc[0].ref_freq))
    wide(12, 4)
    plt.show()

In [ ]:
for (i, site) in enumerate(sites):
    hu.trendplot(a, site, vlines=elines, col='delay_rate')
    plt.title('measured rate [ps/s @ %.0f MHz]' % (a.iloc[0].ref_freq))
    wide(12, 4)
    plt.show()